In [1]:
# Single Dummy Solver. Input is a PBN deal. Output is a DataFrame.

!pip install pandas as pd
!pip install endplay

import pandas as pd

from endplay.dealer import generate_deals
from endplay.types import Deal
from endplay.dds import calc_all_tables


def constraints(deal):
    return True


def generate_single_dummy_deals(predeal_string, produce, env=dict(), max_attempts=1000000, seed=None, show_progress=True, strict=True, swapping=0):
    
    predeal = Deal(predeal_string)

    d = generate_deals(
        constraints,
        predeal=predeal,
        swapping=swapping,
        show_progress=show_progress,
        produce=produce,
        seed=seed,
        max_attempts=max_attempts,
        env=env,
        strict=strict
        )

    d_t = tuple(d) # create a tuple before interop memory goes wonky
    batch_size = 40 # todo: use symbol instead of constant
    t_t = []
    tables = []
    for b in range(0,len(d_t),batch_size):
        batch_tables = calc_all_tables(d_t[b:min(b+batch_size,len(d_t))])
        tables.extend(batch_tables)
        batch_t_t = (tt._data.resTable for tt in batch_tables)
        t_t.extend(batch_t_t)
    return d_t, t_t, tables

In [70]:
def calculate_single_dummy_probabilities(deal, produce=100):

    ns_ew_rows = []
    for ns_ew in ['NS','EW']:
        s = deal[2:].split()
        if ns_ew == 'NS':
            s[1] = '...'
            s[3] = '...'
        else:
            s[0] = '...'
            s[2] = '...'
        predeal_string = 'N:'+' '.join(s)
        #print(f"predeal:{predeal_string}")

        d_t, t_t, tables = generate_single_dummy_deals(predeal_string, produce, show_progress=False)
        assert produce == len(d_t) == len(t_t) == len(tables)

        rows = []
        max_display = 4 # pprint only the first n generated deals
        direction_order = [0,2,3,1]
        suit_order = [3,2,1,0,4]
        for ii,(dd,sd,tt) in enumerate(zip(d_t,t_t,tables)):
            if ii < max_display:
                print(f"Deal:{ii+1} Fixed:{ns_ew} Generated:{ii+1}/{produce}")
                dd.pprint()
                print()
                tt.pprint()
                print()
            nswe_flat_l = [sd[suit][direction] for direction in direction_order for suit in suit_order]
            rows.append([dd.to_pbn()]+nswe_flat_l)

        dd_df = pd.DataFrame(rows,columns=['Deal']+[d+s for d in list('NSWE') for s in list('CDHSN')])
        for d in list('NSWE'):
            for s in list('SHDCN'):
                ns_ew_rows.append([ns_ew,d,s]+dd_df[d+s].value_counts(normalize=True).reindex(range(14), fill_value=0).tolist())
    
    df = pd.DataFrame(ns_ew_rows,columns=['Fixed_Direction','Declarer','Suit']+['Take_'+str(n) for n in range(14)])
    return df

In [71]:
#endplay._dds.SetMaxThreads(24)

In [76]:
produce = 100
pbn_deal = 'N:A98.Q43.J.AKQJ93 KT.AJ8.QT9753.87 QJ73.9752.A64.T6 6542.KT6.K82.542'
df = calculate_single_dummy_probabilities(pbn_deal, produce) # where 100 is number of single dummy samples to generate
display(f"Single dummy trick taking probs after {produce} samples {pbn_deal}",df)

Deal:1 Fixed:NS Generated:1/100
              A98
              Q43
              J
              AKQJ93
54                          KT62
AK86                        JT
KQT8                        97532
875                         42
              QJ73
              9752
              A64
              T6

     ♣  ♦  ♥  ♠ NT
  N  9  4  8  9  8
  S  9  4  8  9  8
  E  3  8  4  4  3
  W  3  8  4  4  3

Deal:2 Fixed:NS Generated:2/100
              A98
              Q43
              J
              AKQJ93
K4                          T652
AKT8                        J6
KT9853                      Q72
7                           8542
              QJ73
              9752
              A64
              T6

     ♣  ♦  ♥  ♠ NT
  N 10  4  8 10 11
  S 10  4  8 10 11
  E  2  9  4  2  2
  W  2  9  4  2  2

Deal:3 Fixed:NS Generated:3/100
              A98
              Q43
              J
              AKQJ93
KT                          6542
AKT86                       J
9832                    

'Single dummy trick taking probs after 100 samples N:A98.Q43.J.AKQJ93 KT.AJ8.QT9753.87 QJ73.9752.A64.T6 6542.KT6.K82.542'

,Fixed_Direction,Declarer,Suit,Take_0,Take_1,Take_2,Take_3,Take_4,Take_5,Take_6,Take_7,Take_8,Take_9,Take_10,Take_11,Take_12,Take_13
0,NS,N,S,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.09,0.25,0.54,0.10,0.00,0.00,0.0
1,NS,N,H,0.00,0.00,0.00,0.00,0.00,0.02,0.03,0.13,0.67,0.15,0.00,0.00,0.00,0.0
2,NS,N,D,0.00,0.01,0.00,0.14,0.50,0.30,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.0
3,NS,N,C,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.66,0.28,0.04,0.00,0.0
4,NS,N,N,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.40,0.44,0.09,0.07,0.00,0.0
5,NS,S,S,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.08,0.26,0.52,0.10,0.00,0.00,0.0
6,NS,S,H,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.21,0.58,0.17,0.00,0.00,0.00,0.0
7,NS,S,D,0.00,0.01,0.02,0.11,0.51,0.29,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.0
8,NS,S,C,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.70,0.24,0.04,0.00,0.0
9,NS,S,N,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.46,0.39,0.10,0.05,0.00,0.0
